<a href="http://www.cosmostat.org/" target="_blank"><img src="http://www.cosmostat.org/wp-content/uploads/2017/07/CosmoStat-Logo_WhiteBK-e1499155861666.png" width=200 align="left" ></a>
<a href="https://joliot.cea.fr/drf/joliot/en/Pages/research_entities/NeuroSpin.aspx" target="_blank"><img src="https://pbs.twimg.com/profile_images/378800000664338392/19732a5beb460c501c201b54fcd74ae6.jpeg" width=70 align="left"></a>
<a href="https://www.cea.fr/" target="_blank"><img src="https://upload.wikimedia.org/wikipedia/commons/9/91/CEA_logo_nouveau.svg" width=86 align="right"></a>

# Hand-On Image Processing with PySAP

> Author: [Samuel Farrens](http://www.cosmostat.org/people/sfarrens)    
> [Where the Earth Meets the Sky workshop](https://indico.nbi.ku.dk/event/1330/), May 2021

<img src="https://github.com/CEA-COSMIC/pysap/raw/master/doc/images/schema.jpg" width=400>



#### Abstract

PySAP (Python Sparse data Analysis Package) is an open-source, multidisciplinary image processing tool developed in collaboration between astrophysicists and biomedical imaging experts at CEA Paris-Saclay through the [COSMIC project](http://cosmic.cosmostat.org/) (see [Farrens et. al (2020)](https://arxiv.org/abs/1910.08465)). The objective of this collaboration being to share the image processing experience gained in different domains with the community.

In this hands-on session I will demonstrate some of the "out of the box" image processing tools available in the PySAP-MRI and PySAP-Astro plugins. In particular, I will focus on some standard problems like denoising or deconvolving galaxy images, and reconstructing subsampled MRI scans of a brain. I will also show how to use some of the modular features of PySAP to design image processing tools for new problems of even different imaging domains.

Throughout the session I will endeavour to provide some basic technical background and present the problems such that it should be possible for everyone in the audience to follow. I will also include links to tutorials for more in-depth explanations of some of these topics for those who are interested.

Finally, I will reserve a few minutes at the end to explain how you can contribute to the development of PySAP.

## Contents

1. [Notebook Set Up](#1-Notebook-Set-Up)
1. [Getting Started with PySAP](#2-Getting-Started-with-PySAP)
1. [PySAP Basics](#3-PySAP-Basics)
1. [ModOpt](#4-ModOpt)
1. [PySAP Plug-Ins](#5-PySAP-Plug-Ins)
1. [Contributing to PySAP](#6-Contributing-to-PySAP)

## 1 Notebook Set Up

### Requirements

In order to follow this hands-on tutorial you need to have the following packages installed.

1. [PySAP](https://github.com/CEA-COSMIC/pysap)
1. [Jupyter](https://jupyter.org/)
1. [tablulate](https://github.com/astanin/python-tabulate)

Details on how install PySAP are provided below, the others can be installed as follows.

```bash
$ pip install jupyter tabulate
```

> Here `$` represents your terminal prompt.


### Set Up

The following cell imports some packages and defines some parameters used throughout the notebook. Be sure to execute these cells before moving on to the next section.

In [ ]:
# Uncomment for Google Colab use only
# !pip install python-pysap

In [ ]:
# Uncomment for Google Colab or Docker use 
# !pip install tabulate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tabulate
import warnings
from matplotlib.pylab import rcParams
from IPython.display import HTML, display

# Default figure size
rcParams['figure.figsize'] = (14.0, 8.0)
# Default image colour map
cmap = 'magma'
# Suppress warnings
warnings.filterwarnings('ignore')

## 2 Getting Started with PySAP

In this section I will walk you through what PySAP is, how to install it and how to check that everything is working as expected.

### What is PySAP?

The **Python Sparse data Analysis Package (PySAP)** is a suite of Python packages designed to tackle image processing problems using sparsity. The suite is currently comprised of

- [PySAP](https://github.com/CEA-COSMIC/pysap): the core package that contains image tranforms including bindings to [Sparse2D](https://github.com/CosmoStat/Sparse2D)
- [ModOpt](https://github.com/CEA-COSMIC/ModOpt): a library of modular tools for solving inverse problems
- [PySAP-MRI](https://github.com/CEA-COSMIC/pysap-mri): tools specifically designed for magenetic resonance imaging problems
- [PySAP-Astro](https://github.com/CEA-COSMIC/pysap-astro): tools specifically designed for astrophysical imaging problems

PySAP is still growing with new plug-ins already in development for applications such as electron tomography.

### Installation

Now that you know what PySAP is, how do you go about installing it? 

If you are on a computer running Linux or macOS with Python $\geq 3.6$ then it should be as simple as

```bash
$ pip install python-pysap
```

this should install the full PySAP suite along with all the required dependencies. Note that if you install the macOS 10.15 wheel you will also need to manually install PyQT5 and the PySAP plug-ins.

```bash
$ pip install pyqt5 pysap-astro pysap-mri
```

If you are running Windows or if the `pip install` fails you can also download the PySAP [Docker](https://www.docker.com/) image as follows

```bash
$ docker pull ceacosmic/pysap
```

which comes with PySAP pre-installed.

> See the [PySAP repository](https://github.com/CEA-COSMIC/pysap#installation) for more installation details and options.

### Running this notebook

Once you have PySAP and the other requirements installed you should be able to run all of the exercises and examples in this notebook without any issues. If you installed PySAP via `pip` then simply run the following command.

```bash
$ jupyter notebook
```

If you installed PySAP via the Docker image then run the following command.

```bash
$ docker run -p 8888:8888 -v ${PWD}:/home ceacosmic/pysap
```

This will run launch Jupyter notebook as usual where you have access to your current working directory but using a Docker container as the backend.

Alternatively this notebook can be run remotely using Google Colab or Binder.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sfarrens/pysap-wtemts-2021/blob/main/Hand-OnPySAP.ipynb)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sfarrens/pysap-wtemts-2021/HEAD?filepath=Hand-OnPySAP.ipynb)


### Checking that everything works

If everyting went to plan you should be able to import `pysap` and print the package information.

In [ ]:
# Import PySAP
import pysap

# Print package info
print(pysap.info())

# Check that the PySAP PySparse bindings were built correctly
try:
    import pysparse
except ImportError:
    print('It looks like PySAP did not install correctly.')
else:
    print('It looks like PySAP is properly installed!')

If you see PySAP v0.0.5 and no error messages you should be good to go!

## 3 PySAP Basics

The [PySAP API documentation](https://python-pysap.readthedocs.io/) includes the full list of PySAP features, here will simply look at a few of the basics.

### PySAP Image Objects

As PySAP is primarily an image processing tool it will come as no surprise that the basic PySAP data object is called `Image`. 

In order to demonstrate how to create a PySAP `Image` object we will need a sample image, so let's import a picture of a racoon from Scipy!

In [ ]:
# Load a sample image from Scipy
from scipy.misc import face

# Make the image monochromatic
racoon = face()[..., 0]

# Check the data type
print(f'racoon is of type {type(racoon)}')

# Display the image
plt.imshow(racoon, cmap='gray')
plt.show()

Now we can create a PySAP `Image`.

In [ ]:
# Create PySAP Image object
pysap_racoon = pysap.Image(data=racoon)

By running `help` on this object you can see the available properties.

In [ ]:
help(pysap_racoon)

Let's have a look at the `show` method.

In [ ]:
# Uncomment the line below to display an interactive PySAP Image.
# pysap_racoon.show()

### Loading sample data

As racoons are not currently the main topic of research for most PySAP tools, it would be convenient to have some more pertinent data. Fortunately, PySAP comes with sample images to play with. In the following cells we will load some sample images and display them.

In [ ]:
# Import function to load sample data
from pysap.data import get_sample_data

# Load sample astro image
astro_image = get_sample_data('astro-fits')
mri_image = get_sample_data('mri-slice-nifti')

# Check the data type
print(f'The data are type {type(astro_image)}')

As you can see the sample data are PySAP `Image` objects.

> <span style="color:#E86868;font-family:verdana">Exercise: Try executing the `show` method for these objects.</span>

In [ ]:
# Execise uncomment the lines below and display the PySAP Image objects.
# astro_image.?
# mri_image.?

For the purposes of this tutorial, it will be more convenient to display the images within the notebook. Note that to do so we can treat these objects are standard Numpy arrays.

In [ ]:
# Display the astro image using matplotlib
plt.imshow(astro_image, cmap=cmap)
plt.show()

In [ ]:
# Display the MR image using matplotlib
plt.imshow(mri_image, cmap=cmap)
plt.show()

### Wavelet transforms

One of the essential tools for sparse image analysis are [wavelets](https://en.wikipedia.org/wiki/Wavelet). Wavelets allow us to decompose images into layers of different spatial frequencies. 

> We don't have time to cover wavelets in more depth in this notebook, but you may want to checkout [this tutorial](https://github.com/CosmoStat/Tutorials/blob/ada/wavelets_1.ipynb) for an introduction.

PySAP includes a host of wavelet transforms that can be applied to various different types of images. In the following cell we will display the list of available transforms.

In [ ]:
# Display the list of PySAP transforms
table = [[index + 1, elem] for index, elem in enumerate(pysap.AVAILABLE_TRANSFORMS)]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

In [ ]:
print(f'PySAP has a total of {len(pysap.AVAILABLE_TRANSFORMS)} transforms available.')

As you can see there quite a few transforms available.

### Starlet transform

Let's have a look at one wavelet in particular, the isotropic undecimated wavelet transform (or Starlet), which well suited for astronomical images (see Starck et al. 2015). In PySAP this is called `LinearWaveletTransformATrousAlgorithm`.

In the following cell we will load this transform and then apply it to our sample astro image.

In [ ]:
# Set number of scales
n_scales = 4

# Load the the starlet transform
transform_name = 'LinearWaveletTransformATrousAlgorithm'
starlet_transform = pysap.load_transform(transform_name)(nb_scale=n_scales, verbose=True)

# Apply transform to sample image
starlet_transform.data = astro_image
starlet_transform.analysis()
astro_image_trans = starlet_transform.analysis_data

# Display the result
fig, axes = plt.subplots(n_scales // 2 + n_scales % 2, 2)
axes = axes.flatten()
if n_scales % 2:
    fig.delaxes(axes[-1])
for index, (ax, scale) in enumerate(zip(axes, astro_image_trans)):
    ax.imshow(scale, cmap=cmap)
    if index == n_scales - 1:
        ax.set_title('Coarse Scale', fontsize=20)
    else:
        ax.set_title(f'Wavelet Scale {index + 1}', fontsize=20)
plt.tight_layout()
plt.show()

As you can see the image has deen decomposed into different spatial frequencies allowing us to analyse each one independently.

> <span style="color:#E86868;font-family:verdana">Exercise: Try increasing the number of scales, what changes?</span>  
> <span style="color:#E86868;font-family:verdana">Exercise: Try using the built-in show method on the `starlet_transform` object.</span>

One of the nicest features of the starlet transform is that we can recover the original image by simply adding the different scales together.

In [ ]:
# Reconstruct original image
astro_image_recons = np.sum(astro_image_trans, axis=0)

# Display the reconstruction
plt.imshow(astro_image_recons, cmap=cmap)
plt.show()

### Other Wavelet Transforms

PySAP also supports [PyWavelets](https://pywavelets.readthedocs.io/) bases such as the [symlet](https://en.wikipedia.org/wiki/Symlet).

In [ ]:
# Set number of scales
n_scales = 2

# Load the the wavelet transform
transform_name = 'sym8'
wavelet_transform = pysap.load_transform(transform_name)(nb_scale=n_scales, verbose=True)

# Apply transform to sample image
wavelet_transform.data = mri_image
wavelet_transform.analysis()
mri_image_trans = wavelet_transform.analysis_data

# Display the result
fig, axes = plt.subplots(n_scales + 1, 3)
axes = axes.flatten()
for ax in axes[1:3]:
    fig.delaxes(ax)
axes[0].imshow(mri_image_trans[0], cmap=cmap)
axes[0].set_title('Coarse Scale', fontsize=20)
for index, (ax, image) in enumerate(zip(axes[3:], mri_image_trans[1:])):
    ax.imshow(image, cmap=cmap)
    if not index % 3:
        ax.set_title(f'Wavelet Scale {index // 3 + 1}', fontsize=20)
plt.tight_layout()
plt.show()

> Note that the symlet is anisotropic so there are multiple images for each scale and here the wavelet scales show increasing spatial frequencies.

The following cell demonstrates how you would reconstruct the original image from the decomposition.

In [ ]:
# Apply transform to decomposed image
wavelet_transform.analysis_data = mri_image_trans
mri_image_recons = wavelet_transform.synthesis()

# Display the result
plt.imshow(mri_image_recons, cmap=cmap)
plt.show()

> In this example we have simply reassigned `wavelet_transform.analysis_data` to itself, in a more realistic application we would perform some operations on this data before performing the `synthesis` operation.

## 4 ModOpt

In this section we will consider linear inverse problems of the form 

$$y = Hx$$

where $y$ is the observed/acquired data, $x$ is the underlying model or true data and $H$ is some form of degredation operator. For image processing problems we can consider $y$ the image one observes and $x$ the image one aims to recover.

For a sparse regularisation problem this typically corresponds to an optimisation problem of the form 

$$\begin{aligned} & \underset{\alpha}{\text{argmin}} & \frac{1}{2}\|y-H\phi\alpha\|_2^2 + \lambda\|\alpha\|_1\end{aligned}$$

where $\alpha$ is a sparse representation of the data, $\lambda$ is a regularisation coefficient and $\phi$ is the sparsity transform (*i.e.* the wavelet transform).

> For more details check out [this tutorial](https://github.com/CosmoStat/Tutorials/blob/ada/sparsity_1.ipynb)

In order to solve this type of problem we need to employ an optimisation algorithm. There are many algorithms out there and they can be applied to a huge variety of problems. ModOpt offers standardised implementations of some well known algorithms with the flexibilty to be applied in different ways. This is done by breaking the above problem (and essentially any other) into modular components that include:

- a gradient (*i.e.* how to solve the first $l2$-norm term, also referred to as the *data fidelty* term)
- a linear operator (*i.e.* how to implement $\phi$, for most PySAP applications this will be a given wavelet transform)
- a proximity operator (*i.e.* how to implement the $l1$-norm above or indeed any other form of regularisation)
- a cost function (*i.e.* how to determine if the algorithm has converged)

These pieces are then put into a given algorithm such as the [Forward-Backward algorithm](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm).


### Linear Regression Example

If you are not familiar with these types of problems or indeed the jargon above, don't worry! We will start by looking at something very familiar just to better illustrate the point; how to fit a line to a some data points (*i.e.* linear regression).

Let's make a plotting function so we can display the results without having to repeat the same matplotlib code constantly.

In [ ]:
# Define a function for making regression plots
def regression_plot(x1, y1, x2=None, y2=None):
    """Plot data points and a proposed fit.
    """

    fig = plt.figure(figsize=(12, 8))
    plt.plot(x1, y1, 'o', color="#19C3F5", label='Data')
    if not all([isinstance(var, type(None)) for var in (x2, y2)]):
        plt.plot(x2, y2, '--', color='#FF4F5B', label='Model')
        plt.title('Best Fit Line', fontsize=20)
    plt.xlabel('x', fontsize=18)
    plt.ylabel('y', fontsize=18)
    plt.legend()
    plt.show()

Now let's define some data points and plot them.

In [ ]:
# Set points in x-axis.
x = np.linspace(0.0, 1.0, 21)

# Set points in y-axis.
y = np.array([0.486, 0.866, 0.944, 1.144, 1.103, 1.202, 1.166, 1.191, 1.124, 1.095, 1.122, 1.102, 1.099, 1.017,
              1.111, 1.117, 1.152, 1.265, 1.380, 1.575, 1.857])

# Plot the points
regression_plot(x, y)

You can probably alredy see by eye the line that should fit these data points, but how do we do it in practice?

Well, this problem can actually be solved analytically*, but here we want to use an optimisation algorithm just for the sake of demonstration.

> *See [this tutorial](https://github.com/CosmoStat/Tutorials/blob/ada/inverse_problems_1.ipynb) for an example.

To that end, let's first define a function for the equation of a polynomial line.

$$y = a_0 + a_1x + a_2x^2 + ... + a_kx^k = \sum_{i=0}^{k}a_ix^i$$

> <span style="color:#E86868;font-family:verdana">Exercise: Implement the equation above in the cell below.</span>  
> <span style="color:#E86868;font-family:verdana">Hint: `a` should be list or numpy array that includes all values from $a_0$ to $a_k$.</span>

In [ ]:
# Exercise: uncomment the cells below and implement the equation for a polynomial line

#def y_func(x, a):
#    return ?

In order to fit this line to our data we need to know the values of $a_i$ and the order of the polynomial. Let's assume it's a third order polynomial and make a bad first guess at the values of $a_i$ and see what happens.

In [ ]:
# Baf first guess at values of a_i
a_guess = np.zeros(4)

# Display the result
regression_plot(x, y, x, y_func(x, a_guess))

Unsurprisingly this is a pretty bad fit to the data. However, if we take another look at our equation $y = \sum_{i=0}^{k}a_ix^i$ we might notice that this looks very similar to our definition of a linear inverse problem $y=Hx$. 

The trick here is note that the model that we want to recover are the values of $a_i$, therefore the operator $H$ is actually comprised of the values of $x$ from our data set. To construct this matrix we simply need all the values of $x$ for each order of our polynomial.

In [ ]:
# Construct matrix H
H = np.array([np.zeros(x.size), x, x ** 2, x ** 3]).T

Now we can define our optimisation problem as follows.

$$\begin{aligned} & \underset{x}{\text{argmin}} & \frac{1}{2}\|y-Hx\|_2^2\end{aligned}$$

> Note that we don't require any specific regularisation for this simple problem.

All that is left to do is break this problem down to the components needed for ModOpt.

Let's start with the gradient. ModOpt contains a predefined gradient of the form 

$$\nabla F(a) = H^{T}(Ha-y)$$

simply called `GradBasic`.

In [ ]:
# Import GradBasic from ModOpt
from modopt.opt.gradient import GradBasic

# Create a gradient object
grad_op = GradBasic(y, lambda x: np.dot(H, x), lambda x: np.dot(H.T, x), verbose=False)

Our particular problem doesn't have a regularisation term so we can simply define an identity operator for the proximity operator.

In [ ]:
# Import IdentityProx from ModOpt
from modopt.opt.proximity import IdentityProx

# Create a proximity object
prox_op = IdentityProx()

ModOpt operators specify their contribution to the total cost, so we simply need to specify which operators are used in our problem to the `costObj`.

In [ ]:
# Import costObj from ModOpt
from modopt.opt.cost import costObj

# Create a cost object
cost = costObj([grad_op, prox_op], verbose=False)

Finally, we need to choose an algorithm to solve our optimisation problem. For this exercise we can use the aforementioned Forward-Backward algorithm.

Let's import it from ModOpt.

In [ ]:
# Import Forward-Backward algorithm from ModOpt
from modopt.opt.algorithms import ForwardBackward

> <span style="color:#E86868;font-family:verdana">Exercise: Check the properties of the ForwardBackward class</span>

In [ ]:
# Exercise check the properties of ForwardBackward algorithm object

# ? 

> <span style="color:#E86868;font-family:verdana">Exercise: Create a ForwardBackward class object. Set a value of $\beta=0.01$, and set `auto_iterate=False` and `verbose=False`.</span>  
> <span style="color:#E86868;font-family:verdana">Hint: Everything you need has already been defined in previous cells.</span>


In [ ]:
# Exercise uncomment the lines below and creat a ForwardBackward algorithm object

#alg = ForwardBackward(?)

Let's run the algorithm and see what happens.

In [ ]:
# Run the algorithm for 500 iteations
alg.iterate(500)

In [ ]:
# Extract the final result
a_final = alg.x_final

# Display the result
regression_plot(x, y, x, y_func(x, a_final))

For the sake of time we have skipped over several subtleties and only looked a very simple problem, but hopefully this has demonstrated how ModOpt can be used to solve a linear inverse problem.

## 5 PySAP Plug-Ins

Now that you are familiar with some of the basic functionality of PySAP and how to solve inverse problems, let's take a look at some of the ways these tools can be combined to solve application specific problems.


### PySAP-Astro

PySAP-Astro aims to provide more user-friendly tools to apply to astromical data.

Let's start with a simple denoising problem. First, we will use a PySAP sample galaxy image called `astro-ngc2997`.

> <span style="color:#E86868;font-family:verdana">Exercise: Complete the cell below to download the sample image.</span>

In [ ]:
# Exercise: Uncomment and complete the following lines to load the sample galaxy image and display it

# Load the galaxy image
# galaxy = 

# Display the image
# plt.imshow(galaxy, cmap=cmap)
# plt.show()

Next, we will artificially add some white Gaussian noise to the image to mimic a particularly noisy observation.

In [ ]:
# Load add_noise function from ModOpt
from modopt.signal.noise import add_noise

# Set noise standard deviation
noise_std = 100
# Add noise to image
obs_galaxy = add_noise(galaxy, sigma=noise_std)

# Display the image
plt.imshow(obs_galaxy, cmap=cmap)
plt.show()

> Note we added quit bit of noise for the sake of visualisation

Finally, we will add the `denoise` function from the PySAP-Astro plug-in and use it to remove the noise we added.

In [ ]:
# Load denoising function from PySAP-Astro
from pysap.plugins.astro.denoising.denoise import denoise

# Set number of wavelet scales
n_scales = 4
# Denoise the image
denoise_galaxy = denoise(obs_galaxy, n_scales=n_scales)

# Display the image
plt.imshow(denoise_galaxy, cmap=cmap)
plt.show()

As you can see (particularly from the outter regions) the noise has been removed with a very simple operation. Behind the scens PyASP-Astro decomposes the image using the starlet transform, then performs a thresholding in the high frequency layes before reconstructing the image.

### PySAP-MRI

Similarly to PySAP-Astro, PySAP-MRI aims to provide more user-friendly tools to apply to MRI data.

For this demonstration we will aim to reconstruct a 2D image of a brain with some masked frequencies. We start by loading the image and corresponding mask.

In [ ]:
# Load the MRI data
mri_data = get_sample_data('2d-mri')

# Load the K-Space Cartesian Mask
mri_mask = get_sample_data("cartesian-mri-mask")

# Display the images
fig, axes = plt.subplots(1, 2)
axes = axes.flatten()
axes[0].imshow(mri_data, cmap=cmap)
axes[1].imshow(mri_mask, cmap=cmap)
plt.show()

Next, we convert the mask into positions in Fourier space and use it to mimic an MRI acquisition.

In [ ]:
from mri.operators import FFT
from mri.operators.utils import convert_mask_to_locations

# Get the locations of the kspace samples
kspace_loc = convert_mask_to_locations(mri_mask.data)

# Generate the subsampled kspace
fourier_op = FFT(samples=kspace_loc, shape=mri_data.shape)
kspace_data = fourier_op.op(mri_data)

# Display the centre of the kspace image
plt.imshow(np.real(kspace_data[220:290, 220:290]), cmap=cmap)
plt.show()

Now if we try simply transform this masked Fourier data into direct space we get a fairly poor reconstruction due to the missing frequencies.

In [ ]:
# Transform kspace data to direct space
mri_data_recons_zero = np.abs(fourier_op.adj_op(kspace_data))

# Display the reconstruction
plt.imshow(mri_data_recons_zero, cmap=cmap)
plt.show()

Instead we can try to use sparsity as a regulariser to reconstruct the image. To so so we will use some ModOpt operators and tools from PySAP-MRI.

In [ ]:
# Import operators from ModOpt
from modopt.opt.proximity import SparseThreshold
from modopt.opt.linear import Identity
# Import tools from PySAP-MRI
from mri.operators import WaveletN
from mri.reconstructors import SingleChannelReconstructor

# Define a linear operator using the symlet transform
linear_op = WaveletN(wavelet_name='sym8', nb_scales=4)
# Define an operator to implement sparse thresholding
regularizer_op = SparseThreshold(Identity(), 2 * 1e-7, thresh_type='soft')
# Define a PySAP-MRI image reconstructor object
reconstructor = SingleChannelReconstructor(
    fourier_op=fourier_op,
    linear_op=linear_op,
    regularizer_op=regularizer_op,
    gradient_formulation='synthesis',
    verbose=1,
)

In [ ]:
# Perform the reconstruction
x_final, *_ = reconstructor.reconstruct(
    kspace_data=kspace_data,
    optimization_alg='fista',
    num_iterations=200,
)

mri_recons = np.abs(x_final)

Let's have a look at the result.

In [ ]:
plt.imshow(mri_recons, cmap=cmap)
plt.show()

Not bad!

## 6 Contributing to PySAP

Hopefully I have given you enough of a taste of PySAP to wet your appetites. The code is fully open source and we would be very happy for anyone to contribute and help us to improve things.

### Contribution Guidelines

We have written a set of [Contribution Guidelines](https://github.com/CEA-COSMIC/pysap/blob/master/CONTRIBUTING.md) to help new developers get started.

For any question not answered there you can [open an issue](https://github.com/CEA-COSMIC/pysap/issues/new/choose) on the repository and we will do our best to help you out.

### New Plug-Ins

Do you have an idea for a new PySAP plug-in? We have a [template repository](https://github.com/CEA-COSMIC/pysap-extplugin) that you can use as the basis.